# Installation Instructions

Download and install miniconda: https://conda.io/miniconda.html

Create new environment with gsshapyorm and jupyter:

```bash
$ conda create -n gsshapyorm -c conda-forge gsshapyorm sqlalchemy<2 jupyter
```

Activate the new environment:

```bash
$ conda activate gsshapyorm
```

Navigate to the folder containing this notebook and start jupyter:

```bash
$ jupyter notebook
```

# Database Setup

You will need a PostgreSQL database with the PostGIS 2.5 extension installed. This can easily created using docker:

```bash
$ docker run --name=gsshapy_postgis --env=POSTGRES_PASSWORD=mysecretpassword -p 5432:5432 -d postgis/postgis:12-2.5
```

After the container is created, create a database named `gsshapy_tutorial` and enable the PostGIS extension on it:

```bash
$ docker exec gsshapy_postgis su postgres -c "createdb gsshapy_tutorial"
$ docker exec gsshapy_postgis su postgres -c "psql -d gsshapy_tutorial -c \"CREATE EXTENSION postgis;\""
````

# Tutorial Data


Download the tutorial data and unzip it: [tutorial-data.zip](https://gsshapy.readthedocs.io/en/latest/_downloads/tutorial-data.zip)

# Create GsshaPy Tables

In [ ]:
from gsshapyorm.lib import db_tools as dbt

# Adjust these per your database setup
username = "postgres"
password = "mysecretpassword"
host = "localhost"
port = "5432"
database = "gsshapy_tutorial"

In [ ]:
# Create gsshapy tables in the database
sqlalchemy_url = dbt.init_postgresql_db(
    host=host,
    port=port,
    username=username,
    password=password,
    database=database,
)
sqlalchemy_url

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import inspect

engine = create_engine(sqlalchemy_url)
inspector = inspect(engine)

# List tables
table_names = sorted([table_name for table_name in inspector.get_table_names(schema='public')])
table_names


# Read a Single File

In [ ]:
from sqlalchemy.orm import Session
from gsshapyorm.orm import ProjectFile

# Update this to the location where you extracted tutorial-data.zip
tutorial_data_dir = '/home/tethysdev/tutorial-data'
filename = 'parkcity.prj'

with Session(engine) as session:
    project_file = ProjectFile()

    # File objects have a read() method that can be used to read the file data into the database tables
    project_file.read(
        session=session,
        directory=tutorial_data_dir,
        filename=filename
    )

    # Each ProjectFile instance is associated with an entry in the proj_project_files table
    print(f'Project File ID: {project_file.id}')

    # Most files are read into multiple related tables and accessible through relationship properties
    # For example, project files are read into the prj_project_files and prj_project_cards tables
    # The "projectCards" property of ProjectFile objects is a relationship property that lists the related ProjectCard objects
    for card in project_file.projectCards:
        print(card)

# Read an Entire GSSHA Model

In [ ]:
# Don't forget to close the session
session = Session(engine)

project_file = ProjectFile()

# The ProjectFile object has methods for reading all the files of a GSSHA model, 
# as well as input and output subsets: readProject(), readInput(), and readOutput()
project_file.readProject(
    session=session,
    directory=tutorial_data_dir,
    projectFileName=filename
)

In [ ]:
# Most files are related to the project file and can be accessed using a relationship property
map_table = project_file.mapTableFile
print(map_table)
for index_map in map_table.indexMaps:
    print(index_map)

for table in map_table.mapTables:
    print(table)
    print(f'  {table.indexMap}')
    for value in table.values:
        print(f'  {value}')

In [ ]:
channel_input_file = project_file.channelInputFile

for link in channel_input_file.streamLinks:
    print(link)

In [ ]:
# Don't forget to close the session when you are done with it
session.close()

# Read with Spatial Data

In [ ]:
from gsshapyorm.orm import ProjectionFile

# Don't forget to close the session
session = Session(engine)

srid = 26912  # UTM Zone 12N
project_file = ProjectFile()

# Add spatial=True to read rasters and vector data into PostGIS raster and geometry fields
# Use the spatialReferenceID parameter to set the coordinate system with an EPSG code
project_file.readProject(
    session=session,
    directory=tutorial_data_dir,
    projectFileName=filename,
    spatial=True,
    spatialReferenceID=srid,
)


In [ ]:
channel_input_file = project_file.channelInputFile

for link in channel_input_file.streamLinks:
    # Notice that the geometry field is now populated with a binary string
    print(f'{link.geometry[:50]}...')

In [ ]:
# Geometry objects provide convenience methods for serializing the data in other formats
for link in channel_input_file.streamLinks:
    print(link)
    # GeoJSON
    print(f'  GeoJSON: {link.getAsGeoJson(session)[:100]}...')
    # WKT
    print(f'  WKT: {link.getAsWkt(session)[:100]}...')

In [ ]:
import pyproj
import numbers


# For coordinate conversion
transformer = pyproj.Transformer.from_crs('epsg:26912', 'epsg:4326', always_xy=True)


def reproject_coords(coords):
    """Reproject any GeoJSON coordinates."""
    if len(coords) < 1:
        return []

    if isinstance(coords[0], numbers.Number):
        from_x, from_y, z = coords
        to_x, to_y = transformer.transform(from_x, from_y)
        return [to_x, to_y, z]

    new_coords = []
    for coord in coords:
        new_coords.append(reproject_coords(coord))
    return new_coords

In [ ]:
import json
import folium

# GeoJSON FeatureCollection template
geojson ={
    "type":"FeatureCollection",
    "features": [],
}

for link in channel_input_file.streamLinks:
    link_geojson = json.loads(link.getAsGeoJson(session))
    link_geojson['coordinates'] = reproject_coords(link_geojson['coordinates'])
    
    link_feature = {
        # "id": link.id,
        "type": "Feature",
        "properties": {},
        "geometry": link_geojson,
    }
        
    geojson['features'].append(link_feature)

map = folium.Map(location=(40.591026,-111.434263), zoom_start=12)
folium.GeoJson(geojson, name="streams").add_to(map)
folium.LayerControl().add_to(map)
map

# PostGIS Functions

https://postgis.net/docs/manual-2.5/PostGIS_Special_Functions_Index.html

In [ ]:
from gsshapyorm.orm import IndexMap

# Most files are related to the project file and can be accessed using a relationship property
map_table = project_file.mapTableFile

for index_map in map_table.indexMaps:
    # Notice that the Raster field is now populated with a binary string: postgis well-known binary raster format
    print(index_map)    

In [ ]:
from sqlalchemy import func

# Enable the GDAL drivers for exporting the rasters
session.execute("SET SESSION postgis.gdal_enabled_drivers = 'ENABLE_ALL';")

# PostGIS Functions Used
# ST_AsGDALRaster: https://postgis.net/docs/manual-2.5/RT_ST_AsGDALRaster.html
# ST_ColorMap: https://postgis.net/docs/manual-2.5/RT_ST_ColorMap.html
# ST_Transform: https://postgis.net/docs/manual-2.5/ST_Transform.html
# ST_AsGeoJSON: https://postgis.net/docs/manual-2.5/ST_AsGeoJSON.html
# ST_Envelope: https://postgis.net/docs/manual-2.5/RT_ST_Envelope.html
query = session.query(
    IndexMap.filename,
    # img
    func.ST_AsGDALRaster(
        func.ST_ColorMap(
            func.ST_Transform(IndexMap.raster, 4326),
            1, 'pseudocolor'
        ),
        'PNG', None, 4326
    ).label('img'),
    # extent
    func.ST_AsGeoJSON(
        func.ST_Envelope(func.ST_Transform(IndexMap.raster, 4326))
    ).label('extent'),
).filter(IndexMap.mapTableFile == map_table)  # Only Index Maps belonging to the current Map Table File

# Make temporary directory to save GeoTIFF rasters
geotiffs = []
for row in query:
    raster_path = row.filename.replace('idx', 'png')
    with open(raster_path, 'wb') as f:
        f.write(row.img)
    geotiffs.append((raster_path, row.extent))
print(geotiffs)

In [ ]:
rmap = folium.Map(location=(40.591026,-111.434263), zoom_start=12)
for geotiff, extent in geotiffs:
    extent = json.loads(extent)
    lat = [c[1] for c in extent["coordinates"][0]]
    lng = [c[0] for c in extent["coordinates"][0]]
    bounds = [[min(lat), min(lng)], [max(lat), max(lng)]]
    print(extent)
    print(bounds)
    img = folium.raster_layers.ImageOverlay(
        name=geotiff,
        image=geotiff,
        bounds=bounds
    )
    img.add_to(rmap)

folium.LayerControl().add_to(rmap)
rmap
    

In [ ]:
# Don't forget to close the session when you are done with it
session.close()

# Export Files

In [ ]:
import os

# The ProjectFile object has methods for writing all the files of a GSSHA model, 
# as well as input and output subsets: writeProject(), writeInput(), and writeOutput()
# Don't forget to close the session

# Here's a common workflow
with Session(engine) as session:
    # Enable the GDAL drivers for exporting the rasters
    session.execute("SET SESSION postgis.gdal_enabled_drivers = 'ENABLE_ALL';")

    # Get a previouly loaded project
    print(f'Project File ID: {project_file.id}')
    project_file = session.query(ProjectFile).get(project_file.id)

    # Modify something
    total_time = project_file.getCard('TOT_TIME')
    print(total_time)
    total_time.value = 3600
    session.commit()
    
    # Write only the input files in preparation for running
    write_dir = os.path.join(os.getcwd(), 'example')
    os.makedirs(write_dir, exist_ok=True)
    project_file.writeInput(
        session=session,
        directory=write_dir,
        name='example',
    )

    print('\n'.join(os.listdir(write_dir)))


In [ ]:
import zipfile
import tempfile
from tethys_dataset_services.engines import GeoServerSpatialDatasetEngine

workspace = 'gssha'
coverage_name = 'elevation'
raster_map_file = '/home/tethysdev/gsshapyorm/notebooks/example/example.ele'
default_style = 'gssha:elevation'

# Get the WKT Projection of the srid
projection_string = 'PROJCS["NAD83 / UTM zone 12N",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-111],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","26912"]]'

gs_engine = GeoServerSpatialDatasetEngine(
    endpoint='http://localhost:8181/geoserver/rest',
    username='admin',
    password='geoserver',
    public_endpoint='http://localhost:8181/geoserver/rest'
)

# create projection file in temp
_, tmp_prj_path = tempfile.mkstemp(suffix='.prj')

with open(tmp_prj_path, 'w') as tmp_prj_file:
    tmp_prj_file.write(projection_string)

# zip raster file and projection file together
_, tmp_zip_path = tempfile.mkstemp(suffix='.zip')

with zipfile.ZipFile(tmp_zip_path, 'w') as tmp_zip_file:
    # NOTE: Give project file and raster file same basename when zipping
    tmp_zip_file.write(tmp_prj_path, coverage_name + '.prj')
    tmp_zip_file.write(raster_map_file, coverage_name)

gs_engine.create_coverage_layer(
    layer_id=f"{workspace}:{coverage_name}",
    coverage_type=gs_engine.CT_GRASS_GRID,
    coverage_file=tmp_zip_path,
    default_style=default_style
)

In [ ]:
import zipfile
import tempfile
from tethys_dataset_services.engines import GeoServerSpatialDatasetEngine

workspace = 'gssha'
coverage_name = 'combo'
raster_map_file = '/home/tethysdev/gsshapyorm/notebooks/example/combo.idx'
default_style = 'gssha:index_map'

# Get the WKT Projection of the srid
projection_string = 'PROJCS["NAD83 / UTM zone 12N",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-111],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","26912"]]'

gs_engine = GeoServerSpatialDatasetEngine(
    endpoint='http://localhost:8181/geoserver/rest',
    username='admin',
    password='geoserver',
    public_endpoint='http://localhost:8181/geoserver/rest'
)

# create projection file in temp
_, tmp_prj_path = tempfile.mkstemp(suffix='.prj')

with open(tmp_prj_path, 'w') as tmp_prj_file:
    tmp_prj_file.write(projection_string)

# zip raster file and projection file together
_, tmp_zip_path = tempfile.mkstemp(suffix='.zip')

with zipfile.ZipFile(tmp_zip_path, 'w') as tmp_zip_file:
    # NOTE: Give project file and raster file same basename when zipping
    tmp_zip_file.write(tmp_prj_path, coverage_name + '.prj')
    tmp_zip_file.write(raster_map_file, coverage_name)

gs_engine.create_coverage_layer(
    layer_id=f"{workspace}:{coverage_name}",
    coverage_type=gs_engine.CT_GRASS_GRID,
    coverage_file=tmp_zip_path,
    default_style=default_style
)

In [ ]:
from jinja2 import Template
from requests.exceptions import RequestException
from sqlalchemy.engine import URL
from tethys_dataset_services.engines import GeoServerSpatialDatasetEngine

workspace = 'gssha'
store_name = 'gsshapy_tutorial'
sql_template_file = '/home/tethysdev/gsshapyorm/sql/stream_nodes.sql'

db_url = URL.create(
    "postgresql",
    username="postgres",
    password="mysecretpassword",
    host="172.17.0.2",
    port="5432",
    database="gsshapy_tutorial",
)

gs_engine = GeoServerSpatialDatasetEngine(
    endpoint='http://localhost:8181/geoserver/rest',
    username='admin',
    password='geoserver',
    public_endpoint='http://localhost:8181/geoserver/rest'
)

# Create PostGIS Store
try:
    gs_engine.create_postgis_store(
        store_id=f"{workspace}:{store_name}",
        host=db_url.host,
        port=db_url.port,
        database=db_url.database,
        username=db_url.username,
        password=db_url.password
    )
except RequestException:
    print(f'PostGIS Store named "{workspace}:{store_name}" already exists. Skipping.')

# Get Layer Name
layer_name = 'stream_nodes'

# Get Default Style Name
default_style = 'point'

srid = 26912
sql_context = {}

with open(sql_template_file, 'r') as sql_template_file:
    text = sql_template_file.read()
    template = Template(text)
    sql = ' '.join(template.render(sql_context).split())

# Create SQL View
gs_engine.create_sql_view_layer(
    store_id=f"{workspace}:{store_name}",
    layer_name=layer_name,
    geometry_type="Geometry",
    srid=srid,
    sql=sql,
    default_style=default_style,
    parameters=(
        {
            'name': 'pid',
            'default_value': '5',
            'regex_validator': '^[0-9]+$'
        },
    )
)